In [44]:
import json
import pandas as pd
import numpy as np
import glob
import os
import re
from tqdm import tqdm

train_example_paths = glob.glob('data/train/*.json')
test_example_paths = glob.glob('data/test/*.json')

train_example_names = [fn.split('.')[0] for fn in os.listdir('data/train')]
test_example_names = [fn.split('.')[0] for fn in os.listdir('data/test')]

metadata = pd.read_csv('data/train.csv')
metadata_train = metadata.loc[metadata.Id.isin(train_example_names)]
metadata_test = metadata.loc[metadata.Id.isin(test_example_names)]

import random

docIdx = metadata_train.Id.values.copy()
random.seed(42)
random.shuffle(docIdx)

train_ratio = 0.8
n_train = int(len(docIdx) * train_ratio)
n_val = len(docIdx) - n_train

train_idx = docIdx[:n_train]
val_idx = docIdx[n_train:]

print(f'train size: {len(train_idx)}')
print(f'val size: {len(val_idx)}')

train size: 15728
val size: 3933


### Utils

In [51]:
def load_test_example_by_name(name):
    doc_path = os.path.join('data/test', name + '.json')
    with open(doc_path) as f:
        data = json.load(f)
    return data

def load_train_example_by_name(name):
    doc_path = os.path.join('data/train', name + '.json')
    with open(doc_path) as f:
        data = json.load(f)
    return data

def delete_file(filename):
    if os.path.exists(filename):
        os.remove(filename)

### Feature Extraction Class

In [46]:
import re

_RE_COMBINE_WHITESPACE = re.compile(r"\s+")

def get_features(s):
    return [(w, t) for w, t in zip(\
            s['TOKEN'].values.tolist(),
            s['TARGET'].values.tolist()                          
        )]

import string
puncs = [c for c in string.punctuation]

def mask_numbers(text):
        # Replace each numeric char with '#'
        
        def repl(m):
            return f" {'#' * len(m.group())} "
        text = re.sub(r'[0-9]+', repl, text)
        return text

def make_single_whitespace(text):
    return _RE_COMBINE_WHITESPACE.sub(" ", text).strip()

class TextFeatureExtractor:
    def __init__(self, *args, **kwargs):
        # Initialize super
        
        # Load parameters
        def_args = dict()
        
        # Extract related arguments
        for k, def_val in def_args.items():
            self.__dict__.update({k: kwargs.get(k, def_val)})

    def transform(self, x):
        x.update({'output': [_df2features(x['output'])]})
        return x

    def _mask_numbers(self, text):
        # Replace each numeric char with #
        
        def repl(m):
            return f" {'#' * len(m.group())} "
        text = re.sub(r'[0-9]+', repl, text)
        return text

    def fit_transform(self, data, train_filenames, val_filenames):
        self.train_filenames = train_filenames
        self.val_filenames = val_filenames

        
        output = {}

        # Process each set
        for setname in ['train', 'val']:
            docs = []
            for f in tqdm(self.__dict__.get(f'{setname}_filenames')):
                df_slice = data[f]

                assert not df_slice['TOKEN'].isnull().any(), 'All tokens must have a value'
                df_slice['TARGET'] = df_slice['TARGET'].fillna('OTHER')
                df_slice['TOKEN'] = df_slice['TOKEN'].values.astype('U')
                df_slice['TOKEN'] = df_slice['TOKEN'].apply(mask_numbers)
                df_slice['TOKEN'] = df_slice['TOKEN'].apply(make_single_whitespace)

                data_slice = get_features(df_slice)
                docs.append(data_slice)
            
            X = [_doc2features(s) for s in tqdm(docs)]
            y = [_doc2labels(s) for s in tqdm(docs)]

            del docs
            
            assert(len(X) == len(y))
            
            output[f'{setname}_data'] = (X, y)
        
        return output

class DocGetter(object):
    def __init__(self, data):
        self.n_doc = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(\
            s['TOKEN'].values.tolist(), 
            s['TARGET'].values.tolist()
                                                                     
        )]
        self.grouped = self.data.groupby('FILENAME').apply(agg_func)
        self.docs = [s for s in self.grouped]

def _text2features(text):
    """Returns a list of examples.
    """
    #words = wordpunct_tokenize(text)
    return [_word2features(words, i) for i in range(len(words))]

def _df2features(df):
    """Returns a list of examples.
    """
    for col in ['TOKEN']:
        assert(col in df.columns)
    
    df['TOKEN'] = df['TOKEN'].values.astype('U')
    df['TOKEN'] = df['TOKEN'].apply(mask_numbers)
    df['TOKEN'] = df['TOKEN'].apply(make_single_whitespace)
    
    agg_func = lambda s: [(w,) for w in s['TOKEN'].values.tolist()]
    feature_list = agg_func(df)
    words = [s for s in feature_list]
    
    return _doc2features(words)

def _word2features(words, i):
    word = words[i]
    if not isinstance(word, str):
        word = word[0]
    
    # isfirstname = fe_isname.is_firstname(word)
    # islastname = fe_isname.is_lastname(word)
    # isname = isfirstname or islastname
    
    # if isfirstname and islastname:
    #     word = '#NAME#'
    # elif isfirstname:
    #     word = '#FIRSTNAME#'
    # elif islastname:
    #     word = '#LASTNAME#'

    digit_count = sum(c=='#' for c in word)
    length = len(word)
    assert length > 0, "All tokens must have length > 0"

    features = {
        #'bias': 1.0,
        #'word.index': i
    }
    
    # Add line & word indices
    # features.update({
    #     'word.lineindex': li,
    #     'word.wordindex': wi,
        
    # })
    
    # if isname:
    #     features.update({
    #     'word.lower()': word
        
    # })
    #else:
    # If all digits
    if word.isdigit():
        features.update({
            'isd': True,
            'dct': digit_count,
            '4dg': digit_count == 4,
            'dgr': 1.0,
            'len': length
        })
    else: # Not all digit
        features.update({
            'isd': False,
            'dgr': digit_count / length,
            'dct': digit_count,
            'len': length,
            'wor': word
        })#'pnc': np.mean(np.array([c in puncs for c in word]))

    if i > 0:
        word_other = words[i-1][0]
        features.update({
            '-1': word_other,
            #'-1:word.isupper()': word1.isupper()
        })
        if i > 1:
            word_other = words[i-2][0]
            features.update({
                '-2': word_other,
                #'-2:word.isupper()': word2.isupper()
            })
            if i > 2:
                word_other = words[i-3][0]
                features.update({
                    '-3': word_other,
                    #'-3:word.isupper()': word_other.isupper()
                })
                if i > 3:
                    word_other = words[i-4][0]
                    features.update({
                        '-4': word_other,
                        #'-4:word.isupper()': word_other.isupper()
                    })
                    if i > 4:
                        word_other = words[i-5][0]
                        features.update({
                            '-5': word_other
                        })
                        if i > 5:
                            word_other = words[i-6][0]
                            features.update({
                                '-6': word_other
                            })
                            
    else:
        features['BOS'] = True
    if i < len(words)-1:
        word_other = words[i+1][0]
        features.update({
            '+1':  word_other,
            #'+1:word.isupper()': word1.isupper()
        })
        if i < len(words)-2:
            word_other = words[i+2][0]
            features.update({
                '+2':  word_other,
                #'+2:word.isupper()': word2.isupper()
            })
            if i < len(words)-3:
                word_other = words[i+3][0]
                features.update({
                    '+3':  word_other,
                    #'+3:word.isupper()': word_other.isupper()
                })
                
                if i < len(words)-4:
                    word_other = words[i+4][0]
                    features.update({
                        '+4':  word_other,
                        #'+4:word.isupper()': word_other.isupper()
                    })
        
        
    else:
        features['EOS'] = True
    return features

def _doc2features(doc):
    """Returns a list of examples.
    """
    words = [(ex[0],) for ex in doc]
    return [_word2features(words, i) for i in range(len(words))]

def _doc2labels(doc):
    return [s[-1] for s in doc]
def _doc2tokens(doc):
    return [s[0] for s in doc]

### Preprocessing Functions

In [47]:
def preprocess_tokenize_doc(doc_json):
    doc_text = ' '.join([clean_text(sec['text']) for sec in doc])
    doc_text = make_single_whitespace(doc_text)

    doc_tokens = doc_text.split(' ')
    return doc_tokens

def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower())

### Load Model

In [48]:
import joblib

model = joblib.load('pipeline_model.joblib')

### Create Submission

In [49]:
feature_extractor = TextFeatureExtractor()

In [57]:
test_preds = []
ids = []
labels = []
for test_id in val_idx[:50]:
    
    # Load and preprocess
    doc = load_train_example_by_name(test_id)
    doc_tokens = preprocess_tokenize_doc(doc)

    # Extract features
    x = {'output': pd.DataFrame({'TOKEN': doc_tokens})}
    x = feature_extractor.transform(x)

    # Predict
    pred = model.predict(x['output'])
    pred = pred[0]
    pred = np.array([int(p) for p in pred])

    # Get corresponding tokens
    pos_pred_idx = [i[0] for i in np.argwhere(pred == 1)]
    pred_tokens = [doc_tokens[i] for i in pos_pred_idx]

    pred_joined = ' '.join(pred_tokens)

    test_preds.append(pred_joined)
    ids.append(test_id)
    labels.append(metadata_train.loc[metadata_train.Id == test_id, 'cleaned_label'].values[0])

sub_df = pd.DataFrame(columns = ['Id', 'PredictionString'])
sub_df['Id'] = ids
sub_df['PredictionString'] = test_preds
sub_df['labels'] = labels

In [58]:
pd.options.display.max_rows = 50
sub_df

,Id,PredictionString,labels
0,fdd86521-c308-42ef-8b05-70de53090360,coastal change analysis program coastal change...,coastal change analysis program
1,3d3a2806-1f3d-4ac6-a288-32357eaa95e6,ibtracs,ibtracs
2,c29ff858-294a-47e4-81e3-9932d30aa321,the alzheimer s disease neuroimaging initiativ...,adni
3,16fdb856-59ea-4dce-aa61-e6b450aca460,the alzheimer s disease neuroimaging initiativ...,adni
4,0c847fd4-5031-4702-9a11-07529cef899f,usda rural urban continuum codes the,rural urban continuum codes
5,bc0c96ef-7f58-4004-8615-6e5f5782af94,,international best track archive for climate s...
6,158d788c-e67b-4dda-bbd1-80fb8f938ccf,the national education longitudinal study of n...,national education longitudinal study
7,29894d91-12ad-4999-9e32-091a92726df5,the north american breeding bird survey bbs in,north american breeding bird survey bbs
8,f70051bf-a763-415b-aa66-97ae57f2efc1,mllw tidal level at noaa tidal station noaa ti...,noaa tidal station
9,f4e954ad-15be-4b9d-858a-9679eedf7cd0,the early childhood longitudinal study timss t...,early childhood longitudinal study


In [30]:
sub_df = pd.DataFrame(columns = ['Id', 'PredictionString'])

In [43]:
sub_df.loc[len(sub_df)] = ['id']

ValueError: cannot set a row with mismatched columns

In [42]:
sub_df

,Id,PredictionString
0,id,pred
1,id,pred
2,id,pred
3,id,pred
4,id,pred
5,id,None
